In [1]:
import pandas as pd
import sys
sys.path.insert(0, '../')
from interface import get_query_emb_batch, load_model

In [2]:
# ckpt_pth = "/home/sondors.igor/documents/ColBERT/best_weights/colbert-172000-finish"
ckpt_pth = "/home/sondors.igor/documents/ColBERT/best_weights/colbert-297010-finish"
doc_maxlen = 300
nbits = 2   # bits определяет количество битов у каждого измерения в семантическом пространстве во время индексации
nranks = 1  # nranks определяет количество GPU для использования, если они доступны
kmeans_niters = 4 # kmeans_niters указывает количество итераций k-means кластеризации; 4 — хороший и быстрый вариант по умолчанию. 

device = "cuda"

checkpoint = load_model(ckpt_pth, doc_maxlen, nbits, kmeans_niters, device)

In [3]:
pth = "/home/sondors.igor/documents/ColBERT/DATASET/unique_NmId_brand_title_charc.parquet"
df_unique = pd.read_parquet(pth)
print(len(df_unique))
df_unique.head()

5057759


,NmId,subjectid,subjectname,name
0,15233171,236,Фигурки,"blue sky Фигурка на полку ""Котенок Клайд"" Высо..."
1,153108241,236,Фигурки,"дом фарфора Фигурка на полку ""Котенок Клайд"" В..."
2,153475651,236,Фигурки,"pavone Фигурка на полку ""Котенок Клайд"" Высота..."
3,138739467,268,Мягкие игрушки,maxitoys Мягкая игрушка мишка / плюшевый медве...
4,98795049,268,Мягкие игрушки,maxitoys Мягкая игрушка медведь / Плюшевый миш...


In [4]:
import numpy as np
import os

def process_chunk_128(df, checkpoint, is_mean=True):
    embs_name = get_query_emb_batch(list(df['name']), checkpoint, batch_size=1000, batch_size2=10000, is_mean=True)
    embs_cols = pd.DataFrame(
                embs_name, 
                columns=[f'emb_{i}' for i in range(128)],
                index=df.index)
    return pd.concat([df['NmId'], embs_cols], axis=1)

def process_chunk_512(df, checkpoint, is_mean=False):
    batch = len(df)
    
    embs_name = get_query_emb_batch(list(df['name']), checkpoint, batch_size=1000, batch_size2=10000, is_mean=False)
    
    embs_name_reshaped = embs_name.reshape(batch, 4, 8, 128) # (batch, 32, 128) --> (batch, 4, 8, 128)
    result = embs_name_reshaped.mean(axis=2) # (batch, 4, 128)
    result_flattened = result.reshape(batch, 4 * 128) # (batch, 4*128)
    
    embs_cols = pd.DataFrame(
                result_flattened, 
                columns=[f'emb_{i}' for i in range(512)],
                index=df.index)
    return pd.concat([df['NmId'], embs_cols], axis=1)

def process_chunk_2048(df, checkpoint, is_mean=False):
    batch = len(df)
    
    embs_name = get_query_emb_batch(list(df['name']), checkpoint, batch_size=1000, batch_size2=10000, is_mean=False)
    
    embs_name_reshaped = embs_name.reshape(batch, 16, 2, 128) # (batch, 16, 2, 128)
    result = embs_name_reshaped.mean(axis=2) # (batch, 16, 128)
    result_flattened = result.reshape(batch, 16 * 128) # (batch, 16*128)
    
    embs_cols = pd.DataFrame(
                result_flattened, 
                columns=[f'emb_{i}' for i in range(2048)],
                index=df.index)
    return pd.concat([df['NmId'], embs_cols], axis=1)
    
def process_df(checkpoint, df_to_process, dst_pth):
    # dst_pth = "/storage/WBX_index/igor/colbert_ckpt_172000_embs_128"
    os.makedirs(dst_pth, exist_ok=True)
    
    df_chunks = np.array_split(df_to_process, 500)
    input_args = ((chunk_id, df) for chunk_id, df in enumerate(df_chunks))
    
    for chunk_id, df in input_args:
        chunk = process_chunk_2048(df, checkpoint)
        chunk.to_parquet(f"{dst_pth}/{chunk_id}.parquet")
        print(f"saved: {dst_pth}/{chunk_id}.parquet\n")
        
# dst_pth = "/storage/WBX_index/igor/colbert_ckpt_172000_embs_4096"
dst_pth = "/home/sondors.igor/documents/ColBERT/colbert_ckpt_297010_embs_2048"
process_df(checkpoint, df_unique, dst_pth)

saved: /home/sondors.igor/documents/ColBERT/colbert_ckpt_297010_embs_2048/0.parquet

saved: /home/sondors.igor/documents/ColBERT/colbert_ckpt_297010_embs_2048/1.parquet

saved: /home/sondors.igor/documents/ColBERT/colbert_ckpt_297010_embs_2048/2.parquet

saved: /home/sondors.igor/documents/ColBERT/colbert_ckpt_297010_embs_2048/3.parquet

saved: /home/sondors.igor/documents/ColBERT/colbert_ckpt_297010_embs_2048/4.parquet

saved: /home/sondors.igor/documents/ColBERT/colbert_ckpt_297010_embs_2048/5.parquet

saved: /home/sondors.igor/documents/ColBERT/colbert_ckpt_297010_embs_2048/6.parquet

saved: /home/sondors.igor/documents/ColBERT/colbert_ckpt_297010_embs_2048/7.parquet

saved: /home/sondors.igor/documents/ColBERT/colbert_ckpt_297010_embs_2048/8.parquet

saved: /home/sondors.igor/documents/ColBERT/colbert_ckpt_297010_embs_2048/9.parquet

saved: /home/sondors.igor/documents/ColBERT/colbert_ckpt_297010_embs_2048/10.parquet

saved: /home/sondors.igor/documents/ColBERT/colbert_ckpt_297010_

OSError: [Errno 28] Error writing bytes to file. Detail: [errno 28] No space left on device

In [18]:
stop_id = 429
# dst_pth = "/storage/WBX_index/igor/colbert_ckpt_172000_embs_4096"
dst_pth = "/home/sondors.igor/documents/ColBERT/colbert_ckpt_297010_embs_2048"
    
df_chunks = np.array_split(df_unique, 500)
input_args = ((chunk_id, df) for chunk_id, df in enumerate(df_chunks))


In [19]:
input_args

<generator object <genexpr> at 0x7f09f82d2040>

In [20]:
for chunk_id, df in input_args:
    if chunk_id <= stop_id:
        print("pass")
    else:
        chunk = process_chunk_2048(df, checkpoint)
        chunk.to_parquet(f"{dst_pth}/{chunk_id}.parquet")
        print(f"saved: {dst_pth}/{chunk_id}.parquet\n")

pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass
pass


In [ ]:
df1 = df_unique[:10].copy()
# df1 = df1.set_index('NmId', verify_integrity=True)
df1

In [ ]:
embs_name = get_query_emb_batch(list(df1['name']), checkpoint, batch_size=1000, batch_size2=10000, is_mean= False)
np.shape(embs_name)

In [ ]:
embs_name_reshaped = embs_name.reshape(10, 16, 2, 128) # (batch, 16, 2, 128)
result = embs_name_reshaped.mean(axis=2) # (batch, 16, 128)
result_flattened = result.reshape(10, 16 * 128) # (batch, 16*128)

print(result_flattened.shape) 

In [ ]:
result_flattened = embs_name.reshape(10, 32 * 128)
print(result_flattened.shape) 

In [ ]:
x = 30/611
x

In [ ]:
x/2 * 2000